<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/kgcnn_jarvis_leaderboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install jarvis_leaderboard

In [5]:
pip install  jarvis_leaderboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pandas-1.2.4-cp39-cp39-manylinux1_x86_64.whl (9.7 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.1
    Uninstalling pandas-2.0.1:
      Successfully uninstalled pandas-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.18 requires pandas>=1.3.0, but you have pandas 1.2.4 which is incompatible.
xarray 2022.12.0 requires pandas>=1.3, but you have pandas 1.2.4 which is incompatible.
plotnine 0.10.1 requires pandas>=1.3.5, but you have pandas 1.2.4 which is incompatible.
mizani 0.8.1 requires pandas>=1.3.5, but you have pandas 1.2.4 which is incompatible.
google-colab 1.0.0 requires pandas~=1.5.3, but you have pandas 1.2.4 which is incompatible.
arviz 0.15.1 requires pandas>=1.3.0, but you have pandas 1.2.

Install kgcnn

In [1]:
import os
if not os.path.exists('gcnn_keras'):
  !git clone https://github.com/aimat-lab/gcnn_keras.git

In [2]:
!pip install -e ./gcnn_keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gcnn_keras
  Preparing metadata (setup.py) ... done
  Attempting uninstall: kgcnn
    Found existing installation: kgcnn 3.0.1
    Can't uninstall 'kgcnn'. No files were found to uninstall.
  Running setup.py develop for kgcnn


might have to restart runtime here in case you can't import kgcnn

In [3]:
import jarvis_leaderboard

In [4]:
import kgcnn

Files will be saved in the output_path i.e. 'Out' directory

In [5]:
!jarvis_populate_data.py --benchmark_file SinglePropertyPrediction-test-exfoliation_energy-dft_3d-AI-mae --output_path=Out

benchmark_file SinglePropertyPrediction-test-exfoliation_energy-dft_3d-AI-mae
dataset dft_3d
output_path Out
property exfoliation_energy
method AI
task SinglePropertyPrediction
id_tag jid
dataset file to be used /usr/local/lib/python3.9/dist-packages/jarvis_leaderboard/dataset/AI/SinglePropertyPrediction/dft_3d_exfoliation_energy.json.zip
Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
100% 40.8M/40.8M [00:02<00:00, 16.6MiB/s]
Loading the zipfile...
Loading completed.
number of training samples 650
number of validation samples 81
number of test samples 81


In [6]:
!rm -r exfoliation_en*

rm: cannot remove 'exfoliation_en*': No such file or directory


In [7]:
ls

gcnn_keras/  Out/  sample_data/


In [8]:
from kgcnn.literature.Schnet import make_crystal_model
from tensorflow.keras.optimizers import Adam
ragged=True
model_config = {
    "name": "Schnet",
    "inputs": [
        {'shape': (None,), 'name': "node_number", 'dtype': 'float32', 'ragged': ragged},
        {'shape': (None, 3), 'name': "node_coordinates", 'dtype': 'float32', 'ragged': ragged},
        {'shape': (None, 2), 'name': "range_indices", 'dtype': 'int64', 'ragged': ragged},
        {'shape': (None, 3), 'name': "range_image", 'dtype': 'int64', 'ragged': ragged},
        {'shape': (3, 3), 'name': "graph_lattice", 'dtype': 'float32', 'ragged': False}
    ],
    "input_embedding": {
        "node": {"input_dim": 95, "output_dim": 64}
    },
    "interaction_args": {
        "units": 128, "use_bias": True, "activation": "kgcnn>shifted_softplus", "cfconv_pool": "sum"
    },
    "node_pooling_args": {"pooling_method": "mean"},
    "depth": 4,
    "gauss_args": {"bins": 25, "distance": 5, "offset": 0.0, "sigma": 0.4}, "verbose": 10,
    "last_mlp": {"use_bias": [True, True, True], "units": [128, 64, 1],
                 "activation": ['kgcnn>shifted_softplus', 'kgcnn>shifted_softplus', 'linear']},
    "output_embedding": "graph",
    "use_output_mlp": False,
    "output_mlp": None,  # Last MLP sets output dimension if None.
}
model = make_crystal_model(**model_config)
model.compile(
    loss="mean_absolute_error",
    optimizer=Adam(learning_rate=1e-04),
    metrics=["mean_absolute_error"],
)

INFO:kgcnn.model.utils:Updated model kwargs:
INFO:kgcnn.model.utils:{'name': 'Schnet', 'inputs': [{'shape': (None,), 'name': 'node_number', 'dtype': 'float32', 'ragged': True}, {'shape': (None, 3), 'name': 'node_coordinates', 'dtype': 'float32', 'ragged': True}, {'shape': (None, 2), 'name': 'range_indices', 'dtype': 'int64', 'ragged': True}, {'shape': (None, 3), 'name': 'range_image', 'dtype': 'int64', 'ragged': True}, {'shape': (3, 3), 'name': 'graph_lattice', 'dtype': 'float32', 'ragged': False}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}}, 'make_distance': True, 'expand_distance': True, 'interaction_args': {'units': 128, 'use_bias': True, 'activation': 'kgcnn>shifted_softplus', 'cfconv_pool': 'sum'}, 'node_pooling_args': {'pooling_method': 'mean'}, 'depth': 4, 'gauss_args': {'bins': 25, 'distance': 5, 'offset': 0.0, 'sigma': 0.4}, 'verbose': 10, 'last_mlp': {'use_bias': [True, True, True], 'units': [128, 64, 1], 'activation': ['kgcnn>shifted_softplus', 'kgcnn>s

We choose one of the SinglePropertyPrediction tasks from the [leaderboard](https://pages.nist.gov/jarvis_leaderboard/AI/SinglePropertyPrediction/)

In [9]:
import pandas as pd
import os
from jarvis.core.atoms import Atoms
from kgcnn.data.crystal import CrystalDataset
import numpy as np

def prepare_data(dirname='exfoliation_en',populated_data_path='Out'):

    id_prop_path=os.path.join(populated_data_path,'id_prop.csv')
    df=pd.read_csv(id_prop_path,header=None)
    df.columns=['id','target']

   
    #train
    train_dir = dirname+'_train'
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    train_cif = os.path.join(train_dir,'CifFiles')
    if not os.path.exists(train_cif):
        os.makedirs(train_cif)
    csv_file = train_dir+'/data.csv'
    print ('train_dir',train_dir)
    f=open(csv_file, "w")
    f.write("file_name,index,label\n")
    targets = []
    for i,ii in df.iterrows():
        pos_path=os.path.join(populated_data_path,ii['id'])
        atoms=Atoms.from_poscar(pos_path)
        pmg = (atoms.pymatgen_converter())
        targets.append(ii['target'])
        fname="file_"+str(ii['id'])+".cif"
        #fname="file_"+str(i)+".cif"
        cif_name = os.path.join(train_cif,fname)
        pmg.to(filename=cif_name, fmt="cif")
        line=fname+','+str(i)+','+str(ii['target'])+'\n'
        f.write(line)
    f.close()
    dataset = CrystalDataset(
        data_directory=train_dir, 
        dataset_name=train_dir, 
        file_name="data.csv", 
        file_directory="CifFiles"
    )
    dataset.prepare_data(file_column_name="file_name", overwrite=True)
    dataset.read_in_memory(label_column_name="label")
    labels = np.expand_dims(dataset.get("graph_labels"), axis=-1)
    dataset.map_list(method="set_range_periodic", max_distance=5.0, max_neighbours=20)
  
    return dataset,labels,df

dataset,labels,df = prepare_data(populated_data_path='Out')



train_dir exfoliation_en_train


INFO:kgcnn.data.exfoliation_en_train:Searching for structure files in 'exfoliation_en_train/CifFiles'
INFO:kgcnn.data.exfoliation_en_train:Read 812 single files.
INFO:kgcnn.data.exfoliation_en_train:... Read .cif file 0 from 812
/usr/local/lib/python3.9/dist-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
INFO:kgcnn.data.exfoliation_en_train:Exporting as dict for pymatgen ...
INFO:kgcnn.data.exfoliation_en_train:Saving structures as .json ...
INFO:kgcnn.data.exfoliation_en_train:Making node features from structure...
INFO:kgcnn.data.exfoliation_en_train:Reading structures from .json ...
INFO:kgcnn.data.exfoliation_en_train: ... read structures 0 from 812


In [10]:
len(dataset),len(labels)

(812, 812)

In [11]:
n_train=650
n_val=81
n_test=81
train_index=np.arange(0,n_train)
val_index=np.arange(n_train,n_train+n_val)
test_index=np.arange(n_train+n_val,n_train+n_val+n_test)

In [12]:
x_train,y_train=dataset[train_index], labels[train_index]
x_val, y_val = dataset[val_index].tensor(model_config["inputs"]), labels[val_index]
x_test, y_test = dataset[test_index].tensor(model_config["inputs"]), labels[test_index]

In [13]:
model.fit(
    x_train.tensor(model_config["inputs"]),y_train,
    shuffle=False,
    batch_size=10,
    epochs=20,
    verbose=2,
)

Epoch 1/20
65/65 - 13s - loss: 98.3313 - mean_absolute_error: 98.3313 - 13s/epoch - 195ms/step
Epoch 2/20
65/65 - 3s - loss: 63.0246 - mean_absolute_error: 63.0246 - 3s/epoch - 53ms/step
Epoch 3/20
65/65 - 6s - loss: 58.8325 - mean_absolute_error: 58.8325 - 6s/epoch - 96ms/step
Epoch 4/20
65/65 - 5s - loss: 56.6790 - mean_absolute_error: 56.6790 - 5s/epoch - 80ms/step
Epoch 5/20
65/65 - 3s - loss: 54.4661 - mean_absolute_error: 54.4661 - 3s/epoch - 54ms/step
Epoch 6/20
65/65 - 6s - loss: 53.6125 - mean_absolute_error: 53.6125 - 6s/epoch - 91ms/step
Epoch 7/20
65/65 - 3s - loss: 51.8939 - mean_absolute_error: 51.8939 - 3s/epoch - 53ms/step
Epoch 8/20
65/65 - 6s - loss: 50.9083 - mean_absolute_error: 50.9083 - 6s/epoch - 96ms/step
Epoch 9/20
65/65 - 4s - loss: 49.7760 - mean_absolute_error: 49.7760 - 4s/epoch - 54ms/step
Epoch 10/20
65/65 - 5s - loss: 49.2880 - mean_absolute_error: 49.2880 - 5s/epoch - 69ms/step
Epoch 11/20
65/65 - 6s - loss: 48.7942 - mean_absolute_error: 48.7942 - 6s/e

In [14]:
val_pred = model.predict(x_val)
test_pred = model.predict(x_test)

3/3 [==============================] - 0s 106ms/step


In [15]:
from sklearn.metrics import mean_absolute_error
print (mean_absolute_error(y_val,val_pred),mean_absolute_error(y_test,test_pred))

46.3512412365572 47.35613476223415


In [16]:
len(test_pred)

81

In [35]:
df_test = df[-n_test:]
f=open('SinglePropertyPrediction-test-exfoliation_energy-dft_3d-AI-mae.csv','w')
f.write('id,prediction\n')
for i in range(len(df_test)):
    #print (i)
    jid = df_test.iloc[i]['id']
    target = df_test.iloc[i]['target']
    #print(jid,target,y_test[i][0],test_pred[i][0])
    line=jid+','+str(test_pred[i][0])+'\n'
    f.write(line)
f.close()

In [36]:
x = pd.read_csv('SinglePropertyPrediction-test-exfoliation_energy-dft_3d-AI-mae.csv')

In [37]:
x

,id,prediction
0,JVASP-12918,170.822100
1,JVASP-2035,74.013300
2,JVASP-13942,64.608246
3,JVASP-278,137.864260
4,JVASP-10173,89.864630
...,...,...
76,JVASP-4364,83.664270
77,JVASP-29480,60.650230
78,JVASP-28375,64.239690
79,JVASP-590,71.237015


In [38]:
import os
cwd=os.getcwd()
os.makedirs('kgcnn_schnet')
os.chdir('kgcnn_schnet')

In [39]:
!cp ../SinglePropertyPrediction-test-exfoliation_energy-dft_3d-AI-mae.csv .

In [48]:
!wget https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/metadata.json

--2023-04-24 19:00:34--  https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/metadata.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391 [text/plain]
Saving to: ‘metadata.json’

metadata.json       100%[===================>]     391  --.-KB/s    in 0s      

2023-04-24 19:00:34 (26.9 MB/s) - ‘metadata.json’ saved [391/391]



In [49]:
from jarvis.db.jsonutils import loadjson,dumpjson
meta = loadjson('metadata.json')

In [50]:
meta

{'model_name': 'ALIGNN',
 'project_url': 'https://www.nature.com/articles/s41524-021-00650-1',
 'date_submitted': '01-14-2023',
 'author_email': 'knc6@nist.gov',
 'database_version': '12-12-2022',
 'team_name': 'JARVIS',
 'time_taken_seconds': 458.17,
 'language': 'python',
 'os': 'linux',
 'software_used': 'jarvis-tools,numpy,scipy,torch,alignn',
 'harware_used': 'nisaba-cluster at NIST, V100 Tesla GPU'}

In [52]:
meta['model_name'] = 'kgcnn_schnet'
meta['project_url'] = 'https://github.com/aimat-lab/gcnn_keras/'
# TODO: Add other details
dumpjson(data=meta,filename='metadata.json')

In [53]:
!wget https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/run.sh

--2023-04-24 19:02:50--  https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/run.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54 [text/plain]
Saving to: ‘run.sh’

run.sh              100%[===================>]      54  --.-KB/s    in 0s      

2023-04-24 19:02:50 (2.53 MB/s) - ‘run.sh’ saved [54/54]



In [54]:
os.chdir(cwd)

In [ ]:
!jarvis_upload.py --your_benchmark_directory  kgcnn_schnet --github_username knc6